# Chatbot Tutorial


## Concept.

### LLM
Large Language Models are advanced machine learning models that excel in a wide range of language-related tasks such as text generation, translation, summarization, question answering, and more, without needing task-specific fine tuning for every scenario.

Modern LLMs are typically accessed through a chat model interface that takes a list of [messages](https://python.langchain.com/docs/concepts/messages/) as input and returns a message as output.


### RAG
Retrieval-Augmented Generation. 
It’s like giving your chatbot a brain full of **searchable knowledge**.
Imagine a chatbot that could tap into a vast library of information and generate creative text. That’s the magic of RAG.

### LangChain

LangChain is a powerful Python-based framework designed to simplify the development of applications powered by large language models (LLMs). It does this by providing a modular and flexible structure that streamlines common NLP tasks and the integration of various AI components.

#### Key Players

- `Chains`: These are essentially pipelines that connect various components within the chatbot architecture. In our case, we’ll construct a chain that seamlessly integrates the user’s query with the retrieval manager (RAG) and subsequently, the response generation stage.
- `Agents`: These are like the workers in your chatbot factory. We will create a specialized agent that acts as a bridge between LangChain and RAG. It receives the user’s query from the chain, transmits it to RAG for information retrieval, and then feeds the retrieved data back into the chain for further processing.
- `Prompts`: LangChain empowers you to design prompts that guide the LLM in crafting the most effective response. For example, a prompt might instruct the LLM to provide a succinct answer to the user’s question or generate a concise summary of a retrieved Wikipedia article.


#### LifeCycle

- `Development`: Build your applications using LangChain's open-source components and third-party integrations. Use LangGraph to build stateful agents with first-class streaming and human-in-the-loop support.
- `Productionization`: Use LangSmith to inspect, monitor and evaluate your applications, so that you can continuously optimize and deploy with confidence.
- `Deployment`: Turn your LangGraph applications into production-ready APIs and Assistants with LangGraph Platform.


#### 🤝🏻 RAG
The synergy between RAG’s information retrieval capabilities and LangChain’s modular structure lays the foundation for constructing a chatbot that leverages the vast and complicated content in any data source (in our case PDF) to deliver informative and creative responses to user queries.


### ChatModels

[docs](https://python.langchain.com/docs/concepts/chat_models/)
Chat models offer a standard set of parameters that can be used to configure the model. These parameters are typically used to control the behavior of the model, such as the temperature of the output, the maximum number of tokens in the response, and the maximum time to wait for a response. Please see the standard parameters section for more details.


#### Interface

[BaseChatModel](https://python.langchain.com/api_reference/core/language_models/langchain_core.language_models.chat_models.BaseChatModel.html)

**Key methods**

- `invoke`: The primary method for interacting with a chat model. It takes a list of messages as input and returns a list of messages as output.
- `stream`: A method that allows you to stream the output of a chat model as it is generated.
- `batch`: A method that allows you to batch multiple requests to a chat model together for more efficient processing.
- `bind_tools`: A method that allows you to bind a tool to a chat model for use in the model's execution context.
- `with_structured_output`: A wrapper around the invoke method for models that natively support structured output.



## 1. Setting Environment 

### 1.1 `pipx` installation
### 1.2 `poetry` installation

`poetry` 명령어가 인식되지 않는다면, poetry 설치 경로 `$PATH` 에 추가

```bash
export PATH="$HOME/.local/bin:$PATH"
``` 

## 2. API Keys

Visit '[Get a Gemini API key](https://ai.google.dev/gemini-api/docs/api-key)' to create an account if you don’t have one.






## 3. create a poetry project

poetry를 사용한 프로젝트 생성 명령어는 다음과 같다.

```bash
poetry new poetry-demo
```
이 명령어는 poetry-demo라는 디렉토리를 만들어주고, 해당 디렉토리는 아래와 같은 구조를 가진다.

```
poetry-demo
├── pyproject.toml
├── README.rst
├── poetry_demo
│   └── __init__.py
└── tests
    ├── __init__.py
    └── test_poetry_demo.py
```

**✔️ `pyproject.toml`**
toml 파일은 중요 요소 중 하나인데, 프로젝트의 의존성을 조율해 주는 파일이기 때문.
`pyproject.toml` 파일을 변경할 필요 없이 아래 command를 통해 수정이 가능. 

```bash
poetry add some-dependencies-you-want
```

`poetry.lock`
항상 같은 의존성 환경에서 개발할 수 있도록 의존성 version locked.
처음 프로젝트 구성 시, 프로젝트에 정의된 의존성 파일들을 설치를 위한 명령어.

```
poetry install
```



--- 

## Build a ChatBot Application!

### Install Dependencies

```
poetry add langchain openai pinecone-client langchain-pinecone langchain-openai python-dotenv pypdf
```

### Set Environment Variables

Create a file named `.env` in your project directory. Remember to keep this file safe — it’ll hold sensitive information like your API keys. We’ll use a Python library called dotenv to access these keys securely.

```
OPENAI_API_KEY={openAI API key}
INDEX_NAME={Pinecone index name}
PINECONE_API_KEY={Pinecone API key}
```

In [ ]:
import os

from dotenv import load_dotenv

load_dotenv()

In [ ]:
os.environ.get("OPENAI_API_KEY")

### STEP 2. Loading the PDF Content

[langchain - document_loaders](https://python.langchain.com/docs/integrations/document_loaders/pypdfloader/)

In [3]:

from langchain_community.document_loaders import PyPDFLoader
import os


def get_working_dir():
    working_dir = "gngsn-llm-lab"
    root = []
    for p in os.getcwd().split("/"):
        root.append(p)
        if (p == working_dir):
            break
    return "/".join(root)


root_dir = get_working_dir()
loader = PyPDFLoader(root_dir + "/data/오가노이드사이언스_증권신고서.pdf")
document = loader.load()

In [ ]:
import pprint

pprint.pp(document[0].metadata)
print(document[0].page_content[:1000])

### STEP 3. Splitting Documents into Chunks

**Understanding Chunk Size and Overlap**

The CharacterTextSplitter takes two important parameters:

- `chunk_size`: This defines the maximum number of characters in each split chunk. A smaller chunk size creates more, but shorter, chunks. A larger chunk size creates fewer, but longer, chunks. It’s crucial to find a good balance when choosing the chunk_size. Extremely small chunks might not contain enough context for the language model to understand properly, whereas very large chunks might become processing bottlenecks.

- `chunk_overlap`: This specifies the number of characters that overlap between consecutive chunks. Overlap helps the language model maintain context between neighboring pieces, especially important for longer sentences that might be split across chunks. We set chunk_overlap to 100 characters. This ensures some overlap between chunks, helping the language model understand the flow of information across splits.

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(document)
print(f"created {len(texts)} chunks")

### Embedding models

[LangChain - Embedding models](https://python.langchain.com/docs/concepts/embedding_models/)

<img src='./img/figure1-1.png' />

- (1) **Embed text as a vector**: Embeddings transform text into a numerical vector representation.
- (2) **Measure similarity**: Embedding vectors can be compared using simple mathematical operations.

[AI-Powered (Vector) Search](https://cameronrwolfe.substack.com/p/the-basics-of-ai-powered-vector-search?utm_source=profile&utm_medium=reader2)

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings

from langchain_google_vertexai import VertexAIEmbeddings

embeddings_model = OpenAIEmbeddings()
embeddings = VertexAIEmbeddings(model="text-embedding-004")
vector_store = InMemoryVectorStore(embeddings)

- **Initializing OpenAI**: We initialize the ChatOpenAI LLM specifying temperature=0. This encourages the model to generate more focused and less creative responses.
- **Core RAG Chain**: In LangChain, RetrievalQA.from_chain_type is a function used to create a RetrievalQA chain, a specific type of chain designed for question answering tasks.


If you want customized answers but no training, use RAG.
If you want a chatbot that improves over time, use RLHF.
If you want full customization, fine-tune your own model.

In [ ]:

from langchain.chains import RetrievalQA

from langchain.chat_models import init_chat_model

chat = init_chat_model("gemini-2.0-flash-001", model_provider="google_vertexai")
# chat = ChatOpenAI(verbose=True, temperature=0, model_name="gpt-3.5-turbo")

qa = RetrievalQA.from_chain_type(
    llm=chat, chain_type="stuff", retriever=vector_store.as_retriever()
)

res = qa.invoke("What are the applications of generative AI according the the paper? Please number each application.")
print(res)

res = qa.invoke("Can you please elaborate more on application number 2?")
print(res)

In [17]:
### 🤖 Create a Simple Chatbot
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA

from langchain.vectorstores import Chroma

chat = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

root_dir = os.path.dirname(os.getcwd())
loader = PyPDFLoader(root_dir + "/data/오가노이드사이언스_증권신고서.pdf")
documents = loader.load()

vectorstore = Chroma.from_documents(documents, GoogleGenerativeAIEmbeddings(model="models/text-embedding-004"))
retriever = vectorstore.as_retriever()

memory = ConversationBufferMemory()
qa = RetrievalQA.from_chain_type(llm=chat, retriever=retriever)

print("Question: What is the document referring to? Please summarize the whole content.")
response = qa.run("What is the document referring to? Please summarize the whole content")
print("Bot:", response)


Question: What is the document referring to? Please summarize the whole content.
Bot: This document outlines regulations and potential restrictions related to participating in the demand forecast for a public offering (IPO).

Here's a summary of the key points:

*   **Restrictions on Participation:** Certain entities are restricted from participating in the demand forecast, including those with conflicts of interest, major shareholders of companies recently underwritten by the lead manager, those currently classified as "unfaithful demand forecast participants," and certain venture capital firms.
*   **"Unfaithful Demand Forecast Participant" Designation:** If an institution is designated as an "unfaithful demand forecast participant" after participating in the current demand forecast, they will be restricted from participating in future demand forecasts and from being allocated IPO shares for a certain period.
*   **Regulations on IPO Pricing:** The document references regulations reg

In [ ]:
from langchain.chains import ConversationalRetrievalChain

qa = ConversationalRetrievalChain.from_llm(
    llm=chat, chain_type="stuff", retriever=retriever
)

chat_history = []

questions = [
    "입력된 문서에 대해 한글로 요약 정리해주세요.",
    "오가노이드의 2024년 11월 매출액이 얼마야?",
    "오가노이드의 2024년 3분기 순이익은 얼마야?",
]

for q in questions:
    response = qa({"question": q, "chat_history": chat_history})

    history = (response["question"], response["answer"])
    chat_history.append(history)

    print("Question:", q)
    print("Bot:", response)


/var/folders/8t/1y7tw5tx0f5gz3rvdmvjbnth0000gn/T/ipykernel_53143/4199014866.py:17: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa({"question": q, "chat_history": chat_history})


Question: 입력된 문서에 대해 한글로 요약 정리해주세요.
Bot: {'question': '입력된 문서에 대해 한글로 요약 정리해주세요.', 'chat_history': [('입력된 문서에 대해 한글로 요약 정리해주세요.', '제공된 문서는 편도 오가노이드가 SARS-CoV-2 감염 시 인간 감염을 상세히 재현할 수 있는 최적화된 연구 플랫폼이라고 설명합니다. 이는 미지의 감염체를 신속하게 식별하고 새로운 치료제의 효과를 평가하는 데 유용합니다. 기존의 세포나 동물 모델이 인간 바이러스 감염을 제대로 재현하지 못하는 반면, 편도 오가노이드는 인체와 유사한 감염 환경을 제공하여 차세대 바이러스 감염 연구 및 약물 평가 플랫폼으로 기대됩니다.')], 'answer': '제공된 문서는 편도 오가노이드가 SARS-CoV-2 감염 시 인간 감염을 상세히 재현할 수 있는 최적화된 연구 플랫폼이라고 설명합니다. 이는 미지의 감염체를 신속하게 식별하고 새로운 치료제의 효과를 평가하는 데 유용합니다. 기존의 세포나 동물 모델이 인간 바이러스 감염을 제대로 재현하지 못하는 반면, 편도 오가노이드는 인체와 유사한 감염 환경을 제공하여 차세대 바이러스 감염 연구 및 약물 평가 플랫폼으로 기대됩니다.'}
Question: 오가노이드의 2024년 11월 매출액이 얼마야?
Bot: {'question': '오가노이드의 2024년 11월 매출액이 얼마야?', 'chat_history': [('입력된 문서에 대해 한글로 요약 정리해주세요.', '제공된 문서는 편도 오가노이드가 SARS-CoV-2 감염 시 인간 감염을 상세히 재현할 수 있는 최적화된 연구 플랫폼이라고 설명합니다. 이는 미지의 감염체를 신속하게 식별하고 새로운 치료제의 효과를 평가하는 데 유용합니다. 기존의 세포나 동물 모델이 인간 바이러스 감염을 제대로 재현하지 못하는 반면, 편도 오가노이드는 인체와 유사한 감염 환경을 제공하여 차세대 바이러스 감염 연구 및 약물 평가 플랫폼으로 기대됩니다.'), ('오가노이드의 20

In [23]:
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains import ConversationalRetrievalChain

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template("""Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:""")

ANSWER_PROMPT_TEMPLATE = PromptTemplate.from_template("""Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:""")

qa = ConversationalRetrievalChain.from_llm(
    llm=chat, chain_type="stuff", retriever=retriever,
    condense_question_prompt=CONDENSE_QUESTION_PROMPT,
    combine_docs_chain_kwargs={"prompt": ANSWER_PROMPT_TEMPLATE},
)

chat_history = []

questions = [
    "입력된 문서에 대해 한글로 요약 정리해주세요.",
    "오가노이드의 2024년 11월 매출액이 얼마야?",
    "오가노이드의 2024년 3분기 순이익은 얼마야?",
]

for q in questions:
    response = qa({"question": q, "chat_history": chat_history})

    history = (response["question"], response["answer"])
    chat_history.append(history)

    print("Question:", q)
    print("Bot:", response)


Question: 입력된 문서에 대해 한글로 요약 정리해주세요.
Bot: {'question': '입력된 문서에 대해 한글로 요약 정리해주세요.', 'chat_history': [('입력된 문서에 대해 한글로 요약 정리해주세요.', '해당 회사의 연구에 따르면 편도 오가노이드는 SARS-CoV-2 감염 시 인체 감염을 상세히 재현하는 최적화된 연구 플랫폼입니다. 이는 미지의 감염체를 신속하게 동정하거나 새로운 치료제의 효능을 평가하는 데 유용합니다. 기존 세포/동물 모델이 인간 바이러스 감염 및 증식에 어려움을 겪는 반면, 편도 오가노이드는 인체와 유사한 감염 환경을 구축하여 차세대 바이러스 감염 연구 및 약물 평가 플랫폼으로 기대됩니다.')], 'answer': '해당 회사의 연구에 따르면 편도 오가노이드는 SARS-CoV-2 감염 시 인체 감염을 상세히 재현하는 최적화된 연구 플랫폼입니다. 이는 미지의 감염체를 신속하게 동정하거나 새로운 치료제의 효능을 평가하는 데 유용합니다. 기존 세포/동물 모델이 인간 바이러스 감염 및 증식에 어려움을 겪는 반면, 편도 오가노이드는 인체와 유사한 감염 환경을 구축하여 차세대 바이러스 감염 연구 및 약물 평가 플랫폼으로 기대됩니다.'}
Question: 오가노이드의 2024년 11월 매출액이 얼마야?
Bot: {'question': '오가노이드의 2024년 11월 매출액이 얼마야?', 'chat_history': [('입력된 문서에 대해 한글로 요약 정리해주세요.', '해당 회사의 연구에 따르면 편도 오가노이드는 SARS-CoV-2 감염 시 인체 감염을 상세히 재현하는 최적화된 연구 플랫폼입니다. 이는 미지의 감염체를 신속하게 동정하거나 새로운 치료제의 효능을 평가하는 데 유용합니다. 기존 세포/동물 모델이 인간 바이러스 감염 및 증식에 어려움을 겪는 반면, 편도 오가노이드는 인체와 유사한 감염 환경을 구축하여 차세대 바이러스 감염 연구 및 약물 평가 플랫폼으로 기대됩니다.'), ('오가노이드의 2024년 11월 매출액이

### Ref
- https://aistudio.google.com/app/apikey
- https://medium.com/@suraj_bansal/build-your-own-ai-chatbot-a-beginners-guide-to-rag-and-langchain-0189a18ec401
- https://wikidocs.net/book/14473